In [1]:
from fawkes.fawkes.protection import Fawkes
from facenet_pytorch import MTCNN
import cv2
from PIL import ImageDraw
import numpy as np
import torch
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import time
import glob
from moviepy.editor import *
from PIL import Image as Img
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Create face detector
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)
print(device)

cuda


In [3]:
# # Load a video
# v_cap = cv2.VideoCapture('./input.mp4')
# v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# destination_name = lambda x: os.path.join('frames', x)

# # Loop through video
# batch_size = 32
# frames = []
# boxes = []
# landmarks = []
# all_frames = []
# probs = []
# for i in tqdm(range(v_len)):
    
#     # Load frame
#     success, frame = v_cap.read()
#     if not success:
#         continue
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = Img.fromarray(frame)
#     frame.save(destination_name('img_{}.jpg'.format(i)))
    
#     # When batch is full, detect faces and reset batch list
#     if len(frames) >= batch_size:
#         batch_boxes, p, batch_landmarks = mtcnn.detect(frames, landmarks=True)
#         boxes.extend(batch_boxes)
#         probs.extend(p)
#         frames = []

In [4]:
image_paths = glob.glob(os.path.join('frames', '*'))
image_paths = [path for path in image_paths if "_cloaked" not in path.split("/")[-1]]
mode = 'high'

In [5]:
print(len(image_paths))

1294


In [ ]:
bsize = 10
processor = Fawkes(feature_extractor="high_extract", gpu=0, batch_size=bsize).run_protection(image_paths, mode=mode, th=0.01, sd=1e9, lr=2,
        max_step=3,
        batch_size=bsize, format='png',
        separate_target=True, debug=True, no_align=True)

Identify 1294 files in the directory
